<a href="https://colab.research.google.com/github/stellaevat/ontology-mapping/blob/main/colab/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pronto transformers datasets evaluate \
&& wget -O doid.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZdOsOrtmNATjuoBPDaIkSTUMyxJXyO2KKC6yw?download=1 \
&& wget -O ncit.obo https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/ETmaJIC0fAlItdsp8WQxS_wBzKN_6x08EZrtsOxVnbzvSg?download=1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00
--2023-10-30 19:41:05--  https://gla-my.sharepoint.com/:u:/g/personal/2526934t_student_gla_ac_uk/EfUC_RdrfZd

In [2]:
import pronto
from transformers import AutoTokenizer

In [3]:
ncit = pronto.Ontology("ncit.obo")
doid = pronto.Ontology("doid.obo")

In [4]:
# Get subsumptions from CSV file to a dictionary

def get_mappings_from_file(filename):
  mappings = {}
  with open(filename) as f:
    for line in f:
      source_id, target_id = line.strip().split(',')
      mappings[source_id] = target_id
  return mappings

In [5]:
doid2ncit_equiv = get_mappings_from_file("doid2ncit_equiv.csv")
ncit2doid_equiv = get_mappings_from_file("ncit2doid_equiv.csv")

# Convert relations to sentences

> Currently considering parents, children & siblings for conceptual reasons, but could also take 'n-hop' appraoch, e.g. 1-hop only with parents and children, or 2-hop to include grandparents, grandchildren and siblings.

> How do I incorporate the desired mapping for training? Should I incorporate both all this AND target info, or too much? Could be SELF + desired relatives instead, or SELF + PARENT + DESIRED PARENT, etc.

In [7]:
# Create sentence from the given entity, containing its direct parents, children & siblings

def get_sentence(source_id, source_onto, target_id, target_onto):
  markers = ["[SUB]", "[/SUB]", "[SUP]", "[/SUP]", "[SIBL]", "[/SIBL]"]
  sub_in, sub_out, sup_in, sup_out, sibl_in, sibl_out = markers

  subsumer = source_onto.get_term(source_id)
  equivalent = target_onto.get_term(target_id)
  supersumer = list(equivalent.superclasses(distance=1, with_self=False))[0]
  siblings = supersumer.subclasses(distance=1, with_self=False)
  siblings.remove(equivalent)

  sentence = [sub_in, subsumer.name, sub_out, sup_in, supersumer.name, sup_out]
  for sibling in siblings:
    sentence.extend([sibl_in, sibling.name, sibl_out])

  return " ".join(sentence)

In [10]:
source_id = "DOID:0014667"
print(get_sentence(source_id, doid, doid2ncit_equiv[source_id], ncit))

[SUB] disease of metabolism [/SUB] [SUP] Non-Neoplastic Disorder by Special Category [/SUP] [SIBL] Non-Communicable Disorder [/SIBL] [SIBL] Epidemic Disorder [/SIBL] [SIBL] Endemic Disorder [/SIBL] [SIBL] Degenerative Disorder [/SIBL] [SIBL] Hamartoma [/SIBL] [SIBL] Hyperplasia [/SIBL] [SIBL] Congenital or Acquired Anatomic Abnormality [/SIBL] [SIBL] Radiation-Related Non-Neoplastic Disorder [/SIBL] [SIBL] Rare Non-Neoplastic Disorder [/SIBL] [SIBL] Inflammatory Disorder [/SIBL]
